>20240218-1424: <br>
>- merged to main, split calculate function x,y
>- 20240220-1029: 
>fix '_hsv' in 'def Mask_n_Draw'
>add 'contourArea' variable 'def Mask_n_Draw'

>TO DO : <br>
>- loop dobot
>- add webcam manual focus
***

In [1]:
##### Library #####
import cv2
import numpy as np
import matplotlib.pyplot as plt

import time
import DobotAPI
import DobotTypes
from DobotControl import DobotControl

In [3]:
##### Function ######

# Main 
def Mask_n_Draw(_hsv, _draw_frame):
    frame = _draw_frame
    contourArea = 500
    # HSV color boundary 0-179
    # HSV boundry list [red_lower, red_upper]
    hsv_red1   = [np.array([ 0, 38, 0]), np.array([ 4, 255, 255])]
    hsv_red2   = [np.array([ 163, 60, 0]), np.array([ 180, 255, 255])]
    hsv_yellow = [np.array([ 21, 50, 0]), np.array([ 40, 255, 255])]
    hsv_green = [np.array([ 42, 52, 0]), np.array([ 79, 255, 255])]
    hsv_blue  = [np.array([ 83, 95, 0]), np.array([ 127, 255, 255])]
    hsv_purple  = [np.array([ 116, 46, 0]), np.array([ 170, 255, 255])]

    # create mask image
    mask_red1   = cv2.inRange(_hsv, hsv_red1[0], hsv_red1[1])
    mask_red2   = cv2.inRange(_hsv, hsv_red2[0], hsv_red2[1])
    mask_yellow = cv2.inRange(_hsv, hsv_yellow[0], hsv_yellow[1])
    mask_green = cv2.inRange(_hsv, hsv_green[0], hsv_green[1])
    mask_blue  = cv2.inRange(_hsv, hsv_blue[0], hsv_blue[1])
    mask_purple  = cv2.inRange(_hsv, hsv_purple[0], hsv_purple[1])

    # merge red1 and red2
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    
    # Contour Retrieval Mode
    # https://docs.opencv.org/3.4/d9/d8b/tutorial_py_contours_hierarchy.html
    CV2_retr = cv2.RETR_TREE # cv2.RETR_TREE, cv2.RETR_LIST, cv2.RETR_EXTERNAL
    # Contour Approximation Method
    # https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
    CV2_approx = cv2.CHAIN_APPROX_SIMPLE # cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE

    contours_red, _ = cv2.findContours(mask_red, CV2_retr, CV2_approx)
    contours_yellow, _ = cv2.findContours(mask_yellow, CV2_retr, CV2_approx)
    contours_green, _ = cv2.findContours(mask_green, CV2_retr, CV2_approx)
    contours_blue, _ = cv2.findContours(mask_blue, CV2_retr, CV2_approx)
    contours_purple, _ = cv2.findContours(mask_purple, CV2_retr, CV2_approx)

    Contour_draw_Rbox(contours_red, contourArea, (0,0,255), 'red', frame)
    Contour_draw_Rbox(contours_yellow, contourArea, (0,255,255), 'yellow', frame)
    Contour_draw_Rbox(contours_green, contourArea, (0,255,0), 'green', frame)
    Contour_draw_Rbox(contours_blue, contourArea, (255,0,0), 'blue', frame)
    Contour_draw_Rbox(contours_purple, contourArea, (255,0,255), 'purple', frame)
    Contour_draw_Claw(contours_red, contourArea, (0,0,255), 'red', frame, 0)
    Contour_draw_Claw(contours_yellow, contourArea, (0,255,255), 'yellow', frame, 1)
    Contour_draw_Claw(contours_green, contourArea, (0,255,0), 'green', frame, 2)
    Contour_draw_Claw(contours_blue, contourArea, (255,0,0), 'blue', frame, 3)
    Contour_draw_Claw(contours_purple, contourArea, (255,0,255), 'purple', frame, 4)

    return frame


# normal box
def Contour_draw_box(Contours, min_area, box_color, text, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame2draw, (x,y), (x+w,y+h), box_color, 2)
            if text != '':
                cv2.putText(frame2draw, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

# rotate box
def Contour_draw_Rbox(Contours, min_area, box_color, text, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt) # center of box(x,y) , size (w,h) , angle
            rotate = rect[2]
            box = cv2.boxPoints(rect)
            box = np.int32(box)

            cv2.drawContours(frame2draw, [box], 0, box_color, 1)
            if text != '':
                cv2.putText(frame2draw, text+f': {rotate:.1f}', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

# draw claw line
def Contour_draw_Claw(Contours, min_area, box_color, text, frame2draw, object_list_number):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            rotate = rect[2]
            xy_cen = np.int32(rect[0])

            wing_12 = (int((box[1][0]+box[2][0])/2),int((box[1][1]+box[2][1])/2))
            wing_03 = (int((box[0][0]+box[3][0])/2),int((box[0][1]+box[3][1])/2))
            cv2.circle(frame2draw, xy_cen, 0, box_color, 10)
            cv2.arrowedLine(frame2draw, xy_cen, wing_12, box_color, 1)
            cv2.arrowedLine(frame2draw, xy_cen, wing_03, box_color, 1)
            if text != '':
                cv2.putText(frame2draw, text+f': {rotate:.1f}', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)
            
            global object_list_status
            global object_list
            temp_list = []

            if object_list_status == True:
                object_list[object_list_number].append(((xy_cen[0],xy_cen[1]), rotate))


In [4]:
# Initiate Dobot
class Robot(DobotControl):
    def __init__(self):
        super().__init__()

if __name__ == '__main__':
    dobot = Robot()
    dobot.setAddr(Robot.search()[0])
    dobot.startRobot()

class DC: # Dobot Camera offset to 0-100 percent
    def __init__(self): # target cam pixel
        self.dobot_min_x =  207.8959 # 'dobot west most'
        self.dobot_max_x =  368.1212 # 'dobot east most'
        self.dobot_min_y = -105.4796 # 'dobot south most'
        self.dobot_max_y =  120.3360 #'dobot north most'
        self.dobot_min_z =  68 # 'dobot bottom most'
        self.dobot_max_z =  145 # 'dobot upper most'
        self.cam_min_x =  84 # 'cam west most'
        self.cam_max_x = 515 # 'cam east most'
        self.cam_min_y =  59 # 'cam north most'
        self.cam_max_y = 359 # 'cam south most'
        self.dobot_dif_x = self.dobot_max_x - self.dobot_min_x # WorkSpace size
        self.dobot_dif_y = self.dobot_max_y - self.dobot_min_y
        self.cam_dif_x = self.cam_max_x - self.cam_min_x
        self.cam_dif_y = self.cam_max_y - self.cam_min_y
        # !!! opencv cam flip Y axis (pixel 0 at top)
        # webcam -> dobot --> rotate 180 degree

    def cam2botX(self,camX,camY):
        camY = (camY - self.cam_min_y) / self.cam_dif_y
        botX = (camY * self.dobot_dif_x) + self.dobot_min_x
        return  botX
    
    def cam2botY(self,camX,camY):
        camX = (camX - self.cam_min_x) / self.cam_dif_x 
        botY = (camX * self.dobot_dif_y) + self.dobot_min_y
        return  botY

### check coordinate logic #######
# aaaa = DC()
# x_ = 400
# y_ = 300
# print(aaaa.cam2botX(x_,y_))
# print(aaaa.cam2botY(x_,y_))
### expect number : cam pixel ___ dobot distance
## B Top Left :  88,  63 ___	214 -98
## Y Top Right: 467,  63 ___	214 102
## R Bot Left :  88, 351 ___ 366 -98
## G Bot Right: 467, 351 ___ 366 102
## P bot right: 400, 300 ___ 300 50
##################################

# Homing Dobot to close to base
dobot.moveTo(155, -12.6, 100, 40) # Absolute Position

['COM3']
COM3 Connect status: DobotConnect_Successfully
GetDeviceVersion: V0.32.1
Initing dobot COM3
Stopped queue successfully


Device version may be not supported: 0.32.1 Dobot[COM3]


COM3 move to 155.0 -12.6 100.0 40.0


In [40]:
##### Main 1 - Realtime Viewing ######
print('>> Phase 1')

object_list_status = False # for list process later

# Read Video/ Cam
# cap = cv2.VideoCapture('UpskillReskill_OpenCV_2023/shibuya.mp4')
cap = cv2.VideoCapture(1)

vid_w = int(cap.get(3))
vid_h = int(cap.get(4))
vid_fps = int(cap.get(cv2.CAP_PROP_FPS))
vid_scale = 1
print(f'video size = {vid_w} x {vid_h}\nframerate = {vid_fps}')

# fix distortion
dist = np.array([[-0.189400, -0.027179, -0.003365, 0.001168, 0.0]])
cameraMatrix = np.array([[878.507989, 0.        , 331.792216],
                         [0.        , 876.710299, 235.020060],
                         [0.        , 0.        , 1.]])
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (vid_w,vid_h), 1, (vid_w,vid_h))

while (cap.isOpened()):
    # read and check for image availability before continue
    check, frame = cap.read()
    if check != True:
        print('>> video ended or not available')
        break

    # pre-process image
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    frame = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale)))
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # backup for last frame from video
    frame_lastClean = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale)))

    # start drawing
    frame_drawed = Mask_n_Draw(frame_hsv, frame)

    # show window
    cv2.imshow('RGB', frame_drawed)
    # cv2.imshow('R', mask_red)
    # cv2.imshow('G', mask_green)
    # cv2.imshow('B', mask_blue)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        print('>> Keyboard Exit')
        break
cap.release()
cv2.destroyAllWindows()

>> Phase 1
video size = 640 x 480
framerate = 30
>> Keyboard Exit


In [41]:
##### Main 2 - Still Image Process ######
print('>> Phase 2')

object_list_status = True
object_list = [[],[],[],[],[]] # R,Y,G,B,P

frame = frame_lastClean
frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

frame_drawed = Mask_n_Draw(frame_hsv, frame)

# amount of object found
obj_red = len(object_list[0])
obj_yellow = len(object_list[1])
obj_green = len(object_list[2])
obj_blue = len(object_list[3])
obj_purple = len(object_list[4])
print(f'Object count\n   red   : {obj_red:>2}\n   yellow : {obj_yellow:>2}\n   green  : {obj_green:>2}\n   blue  : {obj_blue:>2}\n   purple  : {obj_purple:>2}')

print(object_list[0]) # show red location
print(object_list[1]) # Y
print(object_list[2]) # G
print(object_list[3]) # B
print(object_list[4]) # P

# print(object_list) # everything
# print(object_list[0]) # all red
# print(object_list[0][0]) # red 1st
# print(object_list[0][0][0]) # (x,y) in red first
# print(object_list[0][0][1]) # r in red first
# print(object_list[0][0][0][0]) # just x in red first

object_list_merge_xy = []

for i in object_list:
    if not i: # skip if list contain nothing ([] mean false or 0)
        continue
    else:
        for j in i: # for object sequence in specific color
            j_ = []
            j_.append([j[0][0],j[0][1]]) # convert tuple to list

            k = object_list.index(i) # index for color list
            
            object_list_merge_xy.append(j_[0]) # add [x,y] to end of list
            object_list_merge_xy[len(object_list_merge_xy)-1].append(k) # add color index next to x,y
            j_ = []

print(f'\n{object_list_merge_xy}')

cv2.imshow('RGB_Still', frame_drawed)
cv2.waitKey(0)
cv2.destroyAllWindows()

>> Phase 2
Object count
   red   :  0
   yellow :  4
   green  :  1
   blue  :  0
   purple  :  0
[]
[((215, 325), 80.21759033203125), ((362, 314), 9.86580753326416), ((367, 212), 20.136302947998047), ((230, 198), 15.255120277404785)]
[((352, 88), 79.2156982421875)]
[]
[]

[[215, 325, 1], [362, 314, 1], [367, 212, 1], [230, 198, 1], [352, 88, 2]]


In [42]:

### Write Your Code Here ######################################################
# dobot.moveTo(x, y, z, r) # Absolute Position
# dobot.moveInc(dx, dy, dz, dr) # Relative Position
# dobot.setPump(17, 18) # Active Pump
# dobot.resetPump(17, 18) # Deactive Pump
# dobot.suck() # S U C K
# dobot.unsuck # un S U C K
# time.sleep(1) # sleep 1 sec

aaaa = DC()
# object_list [color blog][number in list][0=xy 1=r][0=x 1=y only in xy]
# print('x=',object_list[0][0][0][0], ' y=',object_list[0][0][0][1])
# print(object_list) # everything
# print(object_list[0]) # all red
# print(object_list[0][0]) # red 1st
# print(object_list[0][0][0]) # xy in red first
# print(object_list[0][0][0][0]) # just x in red first

def drop_cup(colorIndex):
    if colorIndex == 0:
        dobot.moveTo(235, -150, aaaa.dobot_max_z, -40) # R
    elif colorIndex == 1:
        dobot.moveTo(300, -150, aaaa.dobot_max_z, -40) # Y
    elif colorIndex == 2:
        dobot.moveTo(365, -150, aaaa.dobot_max_z, -40) # G
    elif colorIndex == 3:
        dobot.moveTo(275, -205, aaaa.dobot_max_z, -40) # B
    elif colorIndex == 4:
        dobot.moveTo(325, -205, aaaa.dobot_max_z, -40) # P
    else :
        dobot.moveTo(300, 0, aaaa.dobot_max_z+50, -40)


# dobot.moveInc(0, 0, 30, 0)
dobot.setPump(17, 18) # blow
dobot.suck() # no blow no suck

for k in object_list_merge_xy:
    print(f'x= {k[0]}, y= {k[1]}')
    x = aaaa.cam2botX(k[0],k[1])
    y = aaaa.cam2botY(k[0],k[1])
    colololor = k[2]
    print(f'x= {x}, y= {y}')

    dobot.moveTo(x, y, aaaa.dobot_max_z, -40) # to x,y
    time.sleep(1)
    dobot.unsuck() # suck
    dobot.moveTo(x, y, aaaa.dobot_min_z, -40) # lower z
    time.sleep(1)

    dobot.moveTo(x, y, aaaa.dobot_max_z, -40) # raise z
    time.sleep(1)
    drop_cup(colololor)
    # dobot.moveTo(270, -140, aaaa.dobot_max_z, 40) # to cup
    time.sleep(1)
    dobot.setPump(17, 18)
    time.sleep(0.1)
    dobot.suck()

dobot.resetPump(17, 18)
# Homing Dobot to close to base
dobot.moveTo(155, -12.6, 100, 40)

# x = aaaa.cam2botX(object_list[0][0][0][0])
# y = aaaa.cam2botY(object_list[0][0][0][1])
# theta = aaaa.cam2botY(object_list[0][0][1])
# dobot.moveTo(x, y, 70, 40) # Absolute Position
# time.sleep(3) # sleep 1 sec

x= 215, y= 325
x= 349.96233266666667, y= -36.844232018561485
COM3 move to 349.96233266666667 -36.844232018561485 145.0 -40.0
COM3 move to 349.96233266666667 -36.844232018561485 68.0 -40.0
COM3 move to 349.96233266666667 -36.844232018561485 145.0 -40.0
COM3 move to 300.0 -150.0 145.0 -40.0
x= 362, y= 314
x= 344.087405, y= 40.17408167053364
COM3 move to 344.087405 40.17408167053364 145.0 -40.0
COM3 move to 344.087405 40.17408167053364 68.0 -40.0
COM3 move to 344.087405 40.17408167053364 145.0 -40.0
COM3 move to 300.0 -150.0 145.0 -40.0
x= 367, y= 212
x= 289.61080300000003, y= 42.79375220417634
COM3 move to 289.61080300000003 42.79375220417634 145.0 -40.0
COM3 move to 289.61080300000003 42.79375220417634 68.0 -40.0
COM3 move to 289.61080300000003 42.79375220417634 145.0 -40.0
COM3 move to 300.0 -150.0 145.0 -40.0
x= 230, y= 198
x= 282.13362233333334, y= -28.9852204176334
COM3 move to 282.13362233333334 -28.9852204176334 145.0 -40.0
COM3 move to 282.13362233333334 -28.9852204176334 68.0 -4

In [31]:
# dobot.setPump(18, 17)
# dobot.suck()
# dobot.unsuck()
# dobot.blow()
# dobot.resetPump(18, 17)
# dobot.moveTo(207.8959, 120.3360, 72, 40) #T L
# dobot.moveTo(368.1212, 120.3360, 72, 40) #T R
# dobot.moveTo(207.8959, -105.4796, 72, 40) #B L
# dobot.moveTo(368.1212, -105.4796, 72, 40) #B R
# dobot.moveTo(155, -12.6, 100, 40)